# Imports

In [ ]:
import tensorflow as tf

import os
import tempfile
import zipfile
import tqdm
import glob

from cral.pipeline import ObjectDetectionPipe
from cral.models.object_detection import RetinanetConfig


# Download Chess dataset

In [ ]:
def download_chess_dataset(dataset_path=tempfile.gettempdir()):
    zip_url = 'https://public.roboflow.ai/ds/uBYkFHtqpy?key=HZljsh2sXY'
    path_to_zip_file = tf.keras.utils.get_file(
        'chess_pieces.zip',
        zip_url,
        cache_dir=dataset_path, 
        cache_subdir='',
        extract=False)
    directory_to_extract_to = os.path.join(dataset_path,'chess_pieces')
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

    images_dir = os.path.join(dataset_path, 'chess_pieces','train')
    annotation_dir = os.path.join(dataset_path, 'chess_pieces','train')

    for image in tqdm.tqdm(glob.glob(os.path.join(images_dir, '*.jpg'))):
        new_name = image.replace('_jpg.rf.', '')
        os.rename(image, new_name)

        annotation = image.replace('.jpg', '.xml')
        new_name = annotation.replace('_jpg.rf.', '')
        os.rename(annotation, new_name)

    return images_dir, annotation_dir

In [ ]:
images_dir, annotation_dir = download_chess_dataset('./')

# SetUp pipeline

In [ ]:
pipe = ObjectDetectionPipe()

pipe.add_data(
    train_images_dir=images_dir,
    train_anno_dir=annotation_dir,
    annotation_format='pascal_voc',
    split=0.2)

pipe.lock_data()

pipe.set_algo(feature_extractor='resnet50', config=RetinanetConfig())

# Train

In [ ]:
from segmind.keras import CheckpointCallback, KerasCallback

keras_cb = KerasCallback()
ckpt_cb = CheckpointCallback(
            snapshot_interval=10,
            snapshot_path='/tmp',
            checkpoint_prefix='retinanet_test')
pipe.train(
    num_epochs=10,
    snapshot_prefix='test_retinanet',
    snapshot_path='/tmp',
    snapshot_every_n=10,
    batch_size=4,
    #steps_per_epoch=100,
    callbacks=[keras_cb, ckpt_cb])

# Inference

## Imports

In [ ]:
from cral.models.object_detection import annotate_image
from PIL import Image
import numpy as np

## Load checkpoint files

In [ ]:
checkpoint_file= '/tmp/test_retinanet_final'
prediction_func= pipe.prediction_model(checkpoint_file= checkpoint_file)

## Predict and annotate Image

In [ ]:
img_path= 'chess_pieces/test/5a35ba2ec3e0d0b2b12b1758a8ac29aa_jpg.rf.280f9940defacbb5d840aef65a9257e5.jpg'

# make prediction
bboxes, scores, labels= prediction_func(img_path)

# draw predictions on image and save
image_array= np.array(Image.open(img_path))
drawn_image= annotate_image(image_array, bboxes, scores, labels, threshold=0.5)
drawn_image.save('image_retinanet.jpeg')